**Note:** The databases used and the codes can be found in my GitHub repository: https://github.com/willneto/Semiarid

## Semiarid Climate (part 1)

In this code, we will develop an analysis of climate conditions in a part of Brazilian territory, namely, the SUDENE area - States of Maranhão, Piauí, Ceará, Rio Grande do Norte, Paraíba, Pernambuco, Alagoas, Sergipe, Bahia e, parcialmente, os Estados de Minas Gerais e do Espírito Santo.

To do that, we need to go through the following path:

- First, using the climates data from INMET, we will create a database with the indicators reflecting the climate conditions from 1981 to 2010.
- Second, we will use an interpolation technique to fill all cities with climate indicators information.
- Third, we will plot maps to show the regions with more semiarid conditions.

In this notebook, we will develop the first path. Here, the main ambition is to show you how to use the data science framework to manipulate data.

**Libraries**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from unidecode import unidecode
import datetime
import glob

## Importing INMET data

**Note:** INMET means National Institute of Meteorology.

The data we obtain can be requested from INMET through the following link (however, we make this data available here):

https://bdmep.inmet.gov.br/

The available data are from each meteorology station; so, there are more than 200 CSV files. Therefore, we need to import it and work to build a single data frame. First, however, the file has a specific format: in the beginning, there is information about the station; next, we can find the climate data. Below, we present an example for you to understand this:

In [2]:
##Station information
pd.read_csv("Solicitado_MES_81_2010_CONVENCIONAL\\dados_82287_M_1981-01-01_2010-12-31.csv",
            encoding= 'ISO-8859-1',
            sep=':',
            nrows=8,
            decimal=',').head()

,Nome,PARNAIBA
0,Codigo Estacao,82287
1,Latitude,-3.08668
2,Longitude,-41.783139
3,Altitude,52.11
4,Situacao,Desativada


In [3]:
##Data
pd.read_csv("Solicitado_MES_81_2010_CONVENCIONAL\\dados_82287_M_1981-01-01_2010-12-31.csv",
            encoding= 'ISO-8859-1',
            sep=';',
            header = 9,
            decimal=',').head()

,Data Medicao,"DIRECAO PREDOMINANTE DO VENTO, MENSAL(Â° (gr))","EVAPORACAO DO PICHE, MENSAL(mm)","EVAPOTRANSPIRACAO POTENCIAL, BH MENSAL(mm)","EVAPOTRANSPIRACAO REAL, BH MENSAL(mm)","INSOLACAO TOTAL, MENSAL(h)","NEBULOSIDADE, MEDIA MENSAL(dÃ©cimos)","NUMERO DE DIAS COM PRECIP. PLUV, MENSAL(nÃºmero)","PRECIPITACAO TOTAL, MENSAL(mm)","PRESSAO ATMOSFERICA AO NIVEL DO MAR, MEDIA MENSAL(mB)","PRESSAO ATMOSFERICA, MEDIA MENSAL(mB)","TEMPERATURA MAXIMA MEDIA, MENSAL(Â°C)","TEMPERATURA MEDIA COMPENSADA, MENSAL(Â°C)","TEMPERATURA MINIMA MEDIA, MENSAL(Â°C)","UMIDADE RELATIVA DO AR, MEDIA MENSAL(%)","VENTO, VELOCIDADE MAXIMA MENSAL(m/s)","VENTO, VELOCIDADE MEDIA MENSAL(m/s)","VISIBILIDADE, MEDIA MENSAL(codigo)",Unnamed: 18
0,1981-01-31,5.0,162.7,NaN,NaN,190.2,5.597701,7.0,31.5,NaN,1004.718391,31.513793,27.553793,23.900000,74.129310,8.1,5.010345,67.796610,NaN
1,1981-02-28,5.0,NaN,NaN,NaN,187.0,4.794872,7.0,62.7,NaN,1003.760256,31.365385,27.510769,23.857692,74.880435,8.1,4.875641,69.038462,NaN
2,1981-03-31,5.0,114.2,NaN,NaN,191.9,5.698925,17.0,528.3,NaN,1005.005376,30.116129,26.489677,23.129032,83.369565,8.3,3.882796,64.193548,NaN
3,1981-04-30,9.0,124.3,NaN,NaN,249.8,3.822222,6.0,74.5,NaN,1004.151111,31.953333,27.484000,22.910000,77.985294,9.1,3.835556,69.672131,NaN
4,1981-05-31,9.0,79.1,NaN,NaN,177.9,6.066667,16.0,320.0,NaN,1005.205556,30.653333,26.652000,22.886667,81.808333,6.6,2.981111,67.741935,NaN


To import each CSV file, we will create two dictionaries - one to receive the station information and the other to receive the climates data. Each station has a number code; therefore, we will use them as the dictionaries keys.

In [4]:
##Dictionaries
inmet_dados_dic = {} #Data
inmet_loc_dic = {} #Information

In [5]:
#Loop - For each file in the folder
for file_folder in glob.glob("Solicitado_MES_81_2010_CONVENCIONAL\*.csv"):
    
    #We're stripping the folder name to just the file name.
    file = file_folder.replace("Solicitado_MES_81_2010_CONVENCIONAL\\",'')
    
    # In some files, there are no pieces of information for the whole period.
    # We can identify which files have this problem through their names.
    # Here, we will import just who has the entire period.
    if (file.split('_')[3]=='1981-01-01') & (file.split('_')[4]=='2010-12-31.csv'):
        
        #Data
        inmet_dados_dic[int(file.split('_')[1])] = pd.read_csv(file_folder,
                                                               encoding= 'ISO-8859-1',
                                                               sep=';',
                                                               header = 9,
                                                               decimal=',')
        #Information
        inmet_loc_dic[int(file.split('_')[1])] = pd.read_csv(file_folder,
                                                             encoding= 'ISO-8859-1',
                                                             sep=':',
                                                             nrows=8,
                                                             decimal=',')

Some imported information came with extra columns and with the information from the column "Data Medicao" as the index, see:

In [6]:
#Example 1:
inmet_dados_dic[83049][["Data Medicao","Unnamed: 17"]].head()

,Data Medicao,Unnamed: 17
1981-01-31,NaN,NaN
1981-02-28,NaN,NaN
1981-03-31,NaN,NaN
1981-04-30,NaN,NaN
1981-05-31,NaN,NaN


In [7]:
#Example 2:
inmet_dados_dic[83492][["Data Medicao","Unnamed: 18"]].head()

,Data Medicao,Unnamed: 18
0,1981-01-31,NaN
1,1981-02-28,NaN
2,1981-03-31,NaN
3,1981-04-30,NaN
4,1981-05-31,NaN


**We will use a loop to go through the dictionary's data to adjust these problems.**

In [8]:
#For each station within the dictionary
for est in inmet_dados_dic.keys():
    
    #If the column "Data Medicao" has been imported as an index
    if "Data Medicao" not in inmet_dados_dic[est].columns:
        inmet_dados_dic[est] = inmet_dados_dic[est].reset_index(drop=False)
    
    #Column Error - "Shift" columns to the left and delete extra column
    if type(inmet_dados_dic[est]["Data Medicao"][0]) != str:
        
        colunas = inmet_dados_dic[est].columns
        
        inmet_dados_dic[est] = inmet_dados_dic[est].reset_index().drop("Unnamed: 17", axis=1)
        
        inmet_dados_dic[est].columns = colunas
        
        inmet_dados_dic[est] = inmet_dados_dic[est].drop("Unnamed: 17", axis=1)
        
    #Delete extra column
    if "Unnamed: 18" in inmet_dados_dic[est].columns:
        inmet_dados_dic[est] = inmet_dados_dic[est].drop("Unnamed: 18", axis=1)

**Adjustments and New Columns:**

Again, we will use a loop to drive inside the dictionary's data to do:

- From the column with the date, we will build a column informing the year of the data. 
- We will turn the column "Data Medicao" into a format datetime.
- We will turn the column "Data Medicao" the index of the data frame.
- In the column "PRECIPITACAO TOTAL, MENSAL(mm)," we will fill the NaN information with the last report.
- Finally, we will create a column with the aridity index by the following formula: $\frac{Total Precipitation}{Potential Evapotranspiration}$

In [9]:
#Para cada estação dentro do dicionário
for est in inmet_dados_dic.keys():
    
    #Creating column with the year
    inmet_dados_dic[est]["Ano Medicao"] = inmet_dados_dic[est]["Data Medicao"].apply(lambda x: x.split('-')[0])
    
    #Turning the format to datetime
    inmet_dados_dic[est]["Data Medicao"] = pd.to_datetime(inmet_dados_dic[est]["Data Medicao"])
    
    #Tornando the column "Data Medicao" the index
    inmet_dados_dic[est] = inmet_dados_dic[est].set_index("Data Medicao")
    
    #Replacing NaN values
    inmet_dados_dic[est]["PRECIPITACAO TOTAL, MENSAL(mm)"
                        ] = inmet_dados_dic[est]["PRECIPITACAO TOTAL, MENSAL(mm)"
                                                ].fillna(method = "ffill")
    
    #Creating a column with the aridity index
    inmet_dados_dic[est]["Aridity Index"] = (inmet_dados_dic[est
                                                               ]["PRECIPITACAO TOTAL, MENSAL(mm)"
                                                                ]/
                                                inmet_dados_dic[est
                                                               ]['EVAPOTRANSPIRACAO POTENCIAL, BH MENSAL(mm)'])

## Indicators for the period 1981 to 2010

Now we will seek to create three indicators:

- Average annual precipitation.
- Average of the Aridity Index per year.
- Total days without rain.

To do that, we will start building a dictionary for each of these indicators. So, therefore, there will be three dictionaries, and their keys will be, again, the code of each station.

**Average annual precipitation**

In [10]:
#Dictionary
media_prec_anual_dic = {}

Inside the subsequent loops, we will pass through each station and select, for the indicator calculation, only the years with complete information.

In [11]:
for est in inmet_dados_dic.keys():
    
    #Only full years
    anos_complet = (inmet_dados_dic[est].
                    groupby("Ano Medicao").
                    count().
                    loc[inmet_dados_dic[est].
                        groupby("Ano Medicao").
                        count()["PRECIPITACAO TOTAL, MENSAL(mm)"]==12].
                    index)
    
    #Passing the value (calculating the average)
    media_prec_anual_dic[est] = inmet_dados_dic[est].groupby("Ano Medicao"
                                                         ).sum().loc[anos_complet #full years
                                                                    ]["PRECIPITACAO TOTAL, MENSAL(mm)"
                                                                     ].mean() #average

**Aridity Index**

In [12]:
#Dictionary
indice_Aridez= {}

The following loop follows the same logic; however, as you will see, we calculate each month's average, and then we calculate the average of the years.

In [13]:
for est in inmet_dados_dic.keys():
    
    #Only full years
    anos_complet = (inmet_dados_dic[est].
                    groupby("Ano Medicao").
                    count().
                    loc[inmet_dados_dic[est].
                        groupby("Ano Medicao").
                        count()["Aridity Index"]==12].
                    index)
    
    #Passing the value (calculating the average)
    indice_Aridez[est] = (inmet_dados_dic[est].groupby("Ano Medicao").mean(). #Average
                          loc[anos_complet]["Aridity Index"].mean()) #Average

**Total days without rain**

In [14]:
#Dictionary
dias_s_chuv= {}

In [15]:
for est in inmet_dados_dic.keys():
    
    #Only full years
    anos_complet = (inmet_dados_dic[est].
                    groupby("Ano Medicao").
                    count().
                    loc[inmet_dados_dic[est].
                        groupby("Ano Medicao").
                        count()["NUMERO DE DIAS COM PRECIP. PLUV, MENSAL(nÃºmero)"]==12].
                    index)
    
    #Passing the value (calculating the average)
    dias_s_chuv[est] = (inmet_dados_dic[est].
                        groupby("Ano Medicao").
                        sum(). #Here, as in the case of average annual precipitation, we add up.
                        loc[anos_complet]["NUMERO DE DIAS COM PRECIP. PLUV, MENSAL(nÃºmero)"].
                        mean()) #Average

**Now, using the dictionaries, we will create a dataframe for each indicator.**

In [16]:
# Average annual precipitation
media_prec_anual_df = pd.DataFrame(media_prec_anual_dic.values(),
                                   columns = ["MEDIA_PREC_ANUAL"],
                                   index=media_prec_anual_dic.keys())

In [17]:
#Result!
media_prec_anual_df.head()

,MEDIA_PREC_ANUAL
82024,1857.982759
82029,NaN
82042,2294.330000
82067,3171.840000
82098,2538.106667


In [18]:
# Aridity index
indice_Aridez_df = pd.DataFrame(indice_Aridez.values(),
                                columns = ["Aridity Index"],
                                index=indice_Aridez.keys())

In [19]:
#Result!
indice_Aridez_df.head()

,Aridity Index
82024,1.067662
82029,NaN
82042,1.285756
82067,NaN
82098,1.449600


In [20]:
# Days without rain
dias_s_chuv_df = pd.DataFrame(dias_s_chuv.values(),
                              columns = ["dias_s_chuv"],
                              index=dias_s_chuv.keys())

In [21]:
#Result!
dias_s_chuv_df.head()

,dias_s_chuv
82024,135.736842
82029,NaN
82042,157.200000
82067,239.500000
82098,194.600000


Days **without** rain!

As it was created, the data is reflecting the days WITH rain, as a year has 365 days we will subtract to identify the days WITHOUT rain.

In [22]:
dias_s_chuv_df["dias_s_chuv"] = (365 - dias_s_chuv_df["dias_s_chuv"])

## Building the final data frame

In [23]:
##Concatenating data frames (We are also deleting the NaN values)
df_final = pd.concat([media_prec_anual_df,indice_Aridez_df,dias_s_chuv_df], axis=1).dropna()

In [24]:
#Result!
df_final.head()

,MEDIA_PREC_ANUAL,Aridity Index,dias_s_chuv
82024,1857.982759,1.067662,229.263158
82042,2294.330000,1.285756,207.800000
82098,2538.106667,1.449600,170.400000
82106,3060.840000,1.937485,129.941176
82113,2492.100000,1.432446,154.666667


**Now we will add the locations of the weather stations (latitude and longitude)**

In [25]:
##Going back to the dictionary we created at the beginning, observe how it is organized:
inmet_loc_dic[82113]

,Nome,BARCELOS
0,Codigo Estacao,82113
1,Latitude,-0.97416666
2,Longitude,-62.9286111
3,Altitude,30.65
4,Situacao,Operante
5,Data Inicial,1981-01-01
6,Data Final,2010-12-31
7,Periodicidade da Medicao,Mensal


In our final data frame, we will create a column for the latitude and another for the longitude. We will then fill in this information by capturing it from the "inmet_loc_dic" dictionary.

In [26]:
#Empty columns
df_final["latitude"] = ''
df_final["longitude"] = ''

We will use a loop to fill these columns that will go through all the stations in our final data frame. Finally, we will manage the station code to locate the information in the dictionary. The following is an example of the logic we will practice to select data from the dictionary.

In [27]:
inmet_loc_dic[82113].set_index("Nome").loc[["Latitude","Longitude"],inmet_loc_dic[82113].set_index("Nome").columns[0]]

Nome
Latitude      -0.97416666
Longitude     -62.9286111
Name:  BARCELOS, dtype: object

In [28]:
#For each information in the DF
for index in df_final.index:
    
    #Capturing the latitude value
    latitude = (inmet_loc_dic[index].set_index("Nome").
                loc[["Latitude","Longitude"],
                    inmet_loc_dic[index].set_index("Nome").columns[0]].values[0])
    
    #Capturing the longitude value
    longitude = (inmet_loc_dic[index].set_index("Nome").
                 loc[["Latitude","Longitude"],
                     inmet_loc_dic[index].set_index("Nome").columns[0]].values[1])
    
    #Passing the value to the final data frame
    df_final.loc[index,"latitude"] = latitude
    df_final.loc[index,"longitude"] = longitude

Columns are like a string; let's convert the values to float.

In [29]:
df_final["latitude"] = df_final["latitude"].astype(float)
df_final["longitude"] = df_final["longitude"].astype(float)

In [30]:
##Result!
df_final.head()

,MEDIA_PREC_ANUAL,Aridity Index,dias_s_chuv,latitude,longitude
82024,1857.982759,1.067662,229.263158,2.829444,-60.661944
82042,2294.330000,1.285756,207.800000,1.832500,-61.120833
82098,2538.106667,1.449600,170.400000,-0.045000,-51.110000
82106,3060.840000,1.937485,129.941176,-0.125278,-67.061111
82113,2492.100000,1.432446,154.666667,-0.974167,-62.928611


## Exporting as a CSV file

In [31]:
df_final.reset_index().to_csv("indicators_81_2010.csv",index=False)